In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.9/287.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 

In [ ]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import gradio as gr


In [ ]:
df = pd.read_csv('train_full.csv')
df = shuffle(df).reset_index()
df


,index,keyword,location,text,target
0,467,nuclear%20reactor,NaN,Check out this awesome profile on #GE's swimmi...,0
1,506,devastated,Chicago; IL,I'm devastated,0
2,587,hijacker,San Francisco Bay Area,Gov. Brown allows parole for 1976 Chowchilla s...,0
3,329,rescued,NIGERIA,Rescued Med migrants arrive in Sicily: Hundre...,1
4,175,flattened,Chorley; Lancashire; UK,Fylde Building set to be flattened: One of Pre...,0
...,...,...,...,...,...
996,76,attacked,Los Angeles; CA,I asked how did he feel attacked by julie. I ...,0
997,832,eyewitness,NaN,DK Eyewitness Travel Guide: Denmark: travel gu...,0
998,399,explode,Oklahoma City; OK,my brain id about to explode lol,0
999,831,collided,NaN,Cyclist who collided with runner on Roanoke gr...,1


In [ ]:
for i in range(len(df)):
  df['text'][i] = df['text'][i].lower()

df

<ipython-input-4-8c9c7480f1fa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i] = df['text'][i].lower()


,index,keyword,location,text,target
0,467,nuclear%20reactor,NaN,check out this awesome profile on #ge's swimmi...,0
1,506,devastated,Chicago; IL,i'm devastated,0
2,587,hijacker,San Francisco Bay Area,gov. brown allows parole for 1976 chowchilla s...,0
3,329,rescued,NIGERIA,rescued med migrants arrive in sicily: hundre...,1
4,175,flattened,Chorley; Lancashire; UK,fylde building set to be flattened: one of pre...,0
...,...,...,...,...,...
996,76,attacked,Los Angeles; CA,i asked how did he feel attacked by julie. i ...,0
997,832,eyewitness,NaN,dk eyewitness travel guide: denmark: travel gu...,0
998,399,explode,Oklahoma City; OK,my brain id about to explode lol,0
999,831,collided,NaN,cyclist who collided with runner on roanoke gr...,1


In [ ]:
vectorizer = TfidfVectorizer()
train_len = (len(df) - 1) - (int((len(df) - 1) * 0.2))
X_train = vectorizer.fit_transform(df['text'][:train_len])
X_test = vectorizer.transform(df['text'][train_len+1:])
y_train = df['target'][:train_len].values
y_test = df['target'][train_len+1:].values

print(len(y_train), len(y_test))
X_train


800 200


<800x3354 sparse matrix of type '<class 'numpy.float64'>'
	with 9338 stored elements in Compressed Sparse Row format>

## Model

In [ ]:
model = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
model.fit(X_train, y_train)

preds = model.predict(X_test)
correct = 0
for i, val in enumerate(preds):
  if y_test[i] == val:
    correct += 1
  
print('Accuracy: ', correct/X_test.shape[0])


Accuracy:  0.675


In [ ]:
def predict(msg: str) -> int:
  global model, vectorizer
  vectorized_msg = vectorizer.transform(pd.DataFrame({'text': [msg]})['text'])
  return model.predict(vectorized_msg)[0]

with gr.Blocks() as demo:
    name = gr.Textbox(label="Message")
    output = gr.Textbox(label="Prediction")
    send_btn = gr.Button("Send")
    send_btn.click(fn=predict, inputs=name, outputs=output, api_name="Send")

demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://72fb5ecdcb317b14c9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://72fb5ecdcb317b14c9.gradio.live


In [1]:
!git init
!git remote add origin https://github.com/WithoutRandom/NPL-Model.git
!git branch -M main
!git push -u origin main

Initialized empty Git repository in /content/.git/
error: refname refs/heads/master not found
fatal: Branch rename failed
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/WithoutRandom/NPL-Model.git'
